In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import os
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix


In [2]:
# Cargar dataset desde CSV
csv_path = "dataset_labels.csv"
df = pd.read_csv(csv_path)

# Codificar etiquetas numéricamente
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["label"])
num_classes = len(label_encoder.classes_)

# Dividir en train y val
train_df = df[df["split"] == "train"]
val_df = df[df["split"] == "val"]
test_df = df[df["split"] == "test"]


# Función para cargar imágenes
def load_images(df, img_size=(224, 224)):  # MobileNetV2 usa 224x224
    images = []
    labels = []
    for _, row in df.iterrows():
        img = cv2.imread(row["image_path"])
        img = cv2.resize(img, img_size)
        img = img.astype("float32") / 255.0  # Normalizar
        images.append(img)
        labels.append(row["label"])
    return np.array(images), to_categorical(labels, num_classes)

# Cargar imágenes
X_train, y_train = load_images(train_df)
X_val, y_val = load_images(val_df)
X_test, y_test = load_images(test_df)  # Agregado para testeo


# Cargar modelo preentrenado MobileNetV2
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Congelar capas preentrenadas

# Construcción del modelo
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.5)(x)
outputs = Dense(num_classes, activation="softmax")(x)
model = Model(inputs=base_model.input, outputs=outputs)

# Compilar el modelo
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [3]:
# Entrenar el modelo
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=5, batch_size=32)

# Evaluar el modelo en el conjunto de testeo
y_pred_prob = model.predict(X_test)
y_pred = np.argmax(y_pred_prob, axis=1)
y_true = np.argmax(y_test, axis=1)

# Calcular métricas
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average="weighted")
recall = recall_score(y_true, y_pred, average="weighted")
f1 = f1_score(y_true, y_pred, average="weighted")

# Matriz de confusión y reporte de clasificación
conf_matrix = confusion_matrix(y_true, y_pred)
report = classification_report(y_true, y_pred)

# Imprimir resultados
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", report)

Epoch 1/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 36s 722ms/step - accuracy: 0.3442 - loss: 1.6494 - val_accuracy: 0.6063 - val_loss: 0.9390
Epoch 2/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 31s 698ms/step - accuracy: 0.5773 - loss: 1.0368 - val_accuracy: 0.6957 - val_loss: 0.7666
Epoch 3/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 30s 688ms/step - accuracy: 0.6718 - loss: 0.7936 - val_accuracy: 0.7271 - val_loss: 0.7001
Epoch 4/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 31s 711ms/step - accuracy: 0.7012 - loss: 0.7173 - val_accuracy: 0.7536 - val_loss: 0.6443
Epoch 5/5
44/44 ━━━━━━━━━━━━━━━━━━━━ 30s 684ms/step - accuracy: 0.7305 - loss: 0.6393 - val_accuracy: 0.7415 - val_loss: 0.6308
6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 688ms/step
Accuracy: 0.6703
Precision: 0.6938
Recall: 0.6703
F1-score: 0.6773
Confusion Matrix:
 [[31 13  6  0]
 [15 32  3  0]
 [ 4  9 36  1]
 [ 2  6  1 23]]
Classification Report:
               precision    recall  f1-score   support

           0       0.60      0.62      0.61        50
           1       0.53      0.64      0.